In [1]:
import numpy as np
import scipy
from qutip import *
from numba import jit, njit

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import plot
import plotly.graph_objs as go

In [2]:
@njit
def adim_free_dyadic(vec_r1, vec_r2, k0):
    a0 = 60. + 1j*0. #space cutoff in nm
    r = k0*( vec_r1 - vec_r2 ) + 1j*0.
    r0 = np.sqrt(np.abs(np.dot(r,r)) ) + 1j*0.
    
    G = np.zeros( shape=(3,3), dtype=np.complex_ )

    r_k_r = ( np.outer(r,r) ) + 1j*0.
    if np.abs(r0) > .00001:

        G = np.exp( 1j*r0 )/(4*np.pi) * ( (1./r0 + 1./r0**2. - 1./r0**3.)*np.eye(3) + (3./r0**3. - 3.*1j/r0**2. - 1./r0)*r_k_r/r0**2.  ) * ( 1. - np.exp(-(np.abs(r0/k0/a0))**4.) )
    
    return G
@njit
def adim_half_space_dyadic(vec_r1, vec_r2, k0_ref, z0, img_active):
    img_A = 0
    if img_active == 1:
        img_A = 1
    
    er_gold = -11.8
    gamma_gold = -0.108 * 1
    epsilon_gold = er_gold*( 1. + 1j*gamma_gold )  #according to chatgpt ?bho?
    reflectance = -(1 - np.sqrt(epsilon_gold))/(1 + np.sqrt(epsilon_gold))
    #reflectance = 1.
    
    k0 = k0_ref + 1j*k0_ref*.0
    
    #z0 = - 2000 + 1j*0   #position of the metallic substrate in nm
    a0 = 10. + 1j*0. #space cutoff in nm
    
    r = k0*( vec_r1 - vec_r2 ) + 1j*0.
    r0 = np.sqrt(np.abs(np.dot(r,r)) ) + 1j*0.
    
    Mrinv = np.diag( np.array([1.+1j*0., 1.+1j*0., -1.+1j*0.]) )
    MZ = np.diag( np.array([0.+1j*0., 0.+1j*0., 1.+1j*0.]) )
    
    plane_pos = np.array([0.+1j*0., 0.+1j*0., z0+1j*0.])
    rI = k0*(vec_r1 - (Mrinv @ (vec_r2+1j*0.) + 2.*plane_pos ) ) + 1j*0.
    rI0 = np.sqrt(np.abs(np.dot(rI,rI)) ) + 1j*0.
    
    r_k_r = ( np.outer(r,r) ) + 1j*0.
    rI_k_rI = ( np.outer(rI,rI) ) + 1j*0.
    
    G = np.zeros( shape=(3,3), dtype=np.complex_ )
    G0 = np.zeros( shape=(3,3), dtype=np.complex_ )
    GI = np.zeros( shape=(3,3), dtype=np.complex_ )
    if np.abs(r0) > .00001:
        Mn0 = - np.exp( 1j*r0 )/(4*np.pi) * ( np.eye(3)/r0**3. - 3.*r_k_r/r0**5. ) #* ( 1. - np.exp(-(np.abs(r0/k0/a0))**4.) )
        Mf0 = np.exp( 1j*r0 )/(4*np.pi)*( np.eye(3)*( 1./r0 + 1j/r0**2.) - r_k_r/r0**2.*( 1./r0 + 3*1j/r0**2.) )
        G0 = Mf0 + Mn0
        
    MnI = - np.exp( 1j*rI0 )/(4*np.pi) * ( np.eye(3)/rI0**3. - 3.*rI_k_rI/rI0**5. )
    MfI = np.exp( 1j*rI0 )/(4*np.pi)*( np.eye(3)*( 1./rI0 + 1j/rI0**2.) - rI_k_rI/rI0**2.*( 1./rI0 + 3*1j/rI0**2.) )
    GI = MfI + MnI - 2.*MZ*np.exp( 1j*rI0 )/(4*np.pi*rI0)
    
    G = G0 - GI*img_A*reflectance
    
    return G

@njit
def system_dyadic_field_amplitude( R, D, k0, dir_k, N_TLS, xvec, yvec, zvec, z0, img_active ):    
    arr_Ex = np.zeros(shape=(xvec.size*yvec.size*zvec.size, N_TLS), dtype=np.complex_)
    arr_Ey = np.zeros(shape=(xvec.size*yvec.size*zvec.size, N_TLS), dtype=np.complex_)
    arr_Ez = np.zeros(shape=(xvec.size*yvec.size*zvec.size, N_TLS), dtype=np.complex_)
    
    arr_E_drive = np.zeros(shape=(xvec.size*yvec.size*zvec.size), dtype=np.complex_)
    
    count = 0
    for nx in range(xvec.size):
        for ny in range(yvec.size):
            for nz in range(zvec.size):
                for n in range(N_TLS):
                    G = adim_half_space_dyadic( np.array([xvec[nx], yvec[ny], zvec[nz]]), R[n], k0, z0, img_active )
                    E = (G + 1j*0.) @ (D[n]+1j*0.)
                    arr_Ex[count][n] = E[0]
                    arr_Ey[count][n] = E[1]
                    arr_Ez[count][n] = E[2]
                    
                arr_E_drive[count] = np.exp( 1j*(k0*dir_k[0]*xvec[nx] + k0*dir_k[1]*yvec[ny] + k0*dir_k[2]*zvec[nz] ) )
                
                count += 1

    return arr_Ex, arr_Ey, arr_Ez, arr_E_drive

@njit
def polar_system_dyadic_field_amplitude( R, D, k0, N_TLS, theta, phi, dist, z0, img_active ):    
    arr_Ex = np.zeros(shape=(theta.size*phi.size, N_TLS), dtype=np.complex_)
    arr_Ey = np.zeros(shape=(theta.size*phi.size, N_TLS), dtype=np.complex_)
    arr_Ez = np.zeros(shape=(theta.size*phi.size, N_TLS), dtype=np.complex_)
    
    count = 0
    for n2 in range(phi.size):
        for n1 in range(theta.size):
            for n in range(N_TLS):
                x = dist * np.sin(phi[n2]) * np.cos(theta[n1])
                y = dist * np.sin(phi[n2]) * np.sin(theta[n1])
                z = dist * np.cos(phi[n2])
                G = adim_half_space_dyadic( np.array([x, y, z]), R[n], k0, z0, img_active )
                E = (G + 1j*0.) @ (D[n]+1j*0.)
                arr_Ex[count][n] = E[0]
                arr_Ey[count][n] = E[1]
                arr_Ez[count][n] = E[2]
            count += 1

    return arr_Ex, arr_Ey, arr_Ez

@njit
def polar_matrix_system_dyadic_field_amplitude( R, D, k0, N_TLS, theta, phi, dist, z0, img_active ):    
    arr_Ex = np.zeros(shape=(theta.size, phi.size, N_TLS), dtype=np.complex_)
    arr_Ey = np.zeros(shape=(theta.size, phi.size, N_TLS), dtype=np.complex_)
    arr_Ez = np.zeros(shape=(theta.size, phi.size, N_TLS), dtype=np.complex_)
    
    for n2 in range(phi.size):
        for n1 in range(theta.size):
            for n in range(N_TLS):
                x = dist * np.sin(phi[n2]) * np.cos(theta[n1])
                y = dist * np.sin(phi[n2]) * np.sin(theta[n1])
                z = dist * np.cos(phi[n2])
                G = adim_half_space_dyadic( np.array([x, y, z]), R[n], k0, z0, img_active )
                E = (G + 1j*0.) @ (D[n]+1j*0.)
                arr_Ex[n1][n2][n] = E[0]
                arr_Ey[n1][n2][n] = E[1]
                arr_Ez[n1][n2][n] = E[2]

    return arr_Ex, arr_Ey, arr_Ez

@njit
def interaction_matrix(R, D, k0, N, z0, img_active):
    K = np.zeros( shape=(N,N), dtype=np.complex_  )
    for n1 in range(N):
        for n2 in range(N):
            G = adim_half_space_dyadic(R[n1], R[n2], k0, z0, img_active)
            K[n1][n2] = np.vdot( (D[n1]+1j*0.) , ( (G + 1j*0.) @ (D[n2] + 1j*0.) ) )

    return K


In [3]:
def uniform_random_lattice(volume_bounds, L, N, max_attempts=10000):
    """
    Generate N points randomly within a specified volume ensuring minimum distance L between points.

    :param volume_bounds: A tuple of 6 elements defining the bounds (x_min, x_max, y_min, y_max, z_min, z_max).
    :param L: Minimum distance between points.
    :param N: Number of points to generate.
    :param max_attempts: Maximum number of attempts to generate each valid point.
    :return: A numpy array of points.
    """
    x_min, x_max, y_min, y_max, z_min, z_max = volume_bounds
    points = []

    def is_valid(point):
        """Check if the point is at least distance L away from all existing points."""
        for p in points:
            if np.linalg.norm(point - p) < L:
                return False
        return True

    while len(points) < N:
        attempts = 0
        while attempts < max_attempts:
            point = np.array([
                np.random.uniform(x_min, x_max),
                np.random.uniform(y_min, y_max),
                np.random.uniform(z_min, z_max)
            ])

            if is_valid(point):
                points.append(point)
                break
            else:
                attempts += 1

        if attempts >= max_attempts:
            print(f"Reached max attempts for a point. Generated {len(points)} points out of {N} requested.")
            break
    
    R = np.array(points)  
    N_TLS = R.shape[0]
    
    Rc = np.sum(R, axis=0)/N_TLS
    R[:,0] -= Rc[0]
    R[:,1] -= Rc[1]
    R[:,2] -= Rc[2]
    
    return R, N_TLS
def ordered_chain(L, N):
    R = np.zeros(shape=(N,3))
    for n in range(N):
        R[n][0] = n*L
    
    Rc = np.sum(R, axis=0)/N
    R[:,0] -= Rc[0]
    R[:,1] -= Rc[1]
    R[:,2] -= Rc[2]
    return R 

def all_ordered_dipole(N, direction):
    D = np.zeros(shape=(N,3))
    for n in range(N):
        D[n][direction] = 1
    return D   
def partially_random_dipoles(N, direction, sigma_rel):
    D = np.zeros(shape=(N,3))
    sigma = np.pi * sigma_rel
    for n in range(N):
        if direction == 0:
            angle_1 = sigma*np.random.normal(0.0, 1.0)
            angle_2 = np.pi/2. + sigma*np.random.normal(0.0, 1.0)
        elif direction == 1:
            angle_1 = np.pi/2. + sigma*np.random.normal(0.0, 1.0)
            angle_2 = np.pi/2. + sigma*np.random.normal(0.0, 1.0)
        elif direction == 2:
            angle_1 = sigma*np.random.normal(0, 1.0)
            angle_2 = sigma*np.random.normal(0, 1.0)    
        D[n][0] = np.cos( angle_1 )*np.sin(angle_2)
        D[n][1] = np.sin( angle_1 )*np.sin(angle_2)
        D[n][2] = np.cos(angle_2)
        
    return D


def plot_points_with_vectors(points, vectors, l0, freq):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Create a custom colormap
    colors = [(0, 0, 1), (0, 0, 0), (1, 0, 0)]  # Blue -> Black -> Red
    cmap_name = 'blue_black_red'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=256)
    #cmap = cm.coolwarm

    
    # Normalize the frequency array to the range [0, 1] for colormap
    norm = plt.Normalize(freq.min(), freq.max())


    sc = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=freq, cmap=cm, marker='o', norm=norm)
    xmax = np.amax(points[:, 0])
    xmin = np.amin(points[:, 0])
    if xmax-xmin <= l0:
        xmax = l0
        xmin = -l0
    ymax = np.amax(points[:, 1])
    ymin = np.amin(points[:, 1])
    if ymax-ymin <= l0:
        ymax = l0
        ymin = -l0
    zmax = np.amax(points[:, 2])
    zmin = np.amin(points[:, 2])
    if zmax-zmin <= l0:
        zmax = l0
        zmin = -l0
    Lmax = np.amax( np.array([xmax,ymax,zmax]) )
    Lmin = np.amin( np.array([xmin,ymin,zmin]) )
    ax.set_xlim(Lmin, Lmax)
    ax.set_ylim(Lmin, Lmax)
    ax.set_zlim(Lmin, Lmax)

    # Plot vectors as arrows
    for point, vector in zip(points, vectors):
        ax.quiver(point[0], point[1], point[2], vector[0], vector[1], vector[2], length=l0/2., normalize=True)

    ax.set_xlabel('X - nm')
    ax.set_ylabel('Y - nm')
    ax.set_zlabel('Z - nm')
    # Add color bar to indicate the frequency values
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Frequency - GHz')


    plt.title(f'Positions of the {len(points)}-TLS with relative frequency and dipole direction')
    plt.show()  
def plot_points_with_vectors_substrate(points, vectors, l0, freq, z0):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Create a custom colormap
    colors = [(0, 0, 1), (0, 0, 0), (1, 0, 0)]  # Blue -> Black -> Red
    cmap_name = 'blue_black_red'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=256)
    
    # Normalize the frequency array to the range [0, 1] for colormap
    norm = plt.Normalize(freq.min(), freq.max())

    # Scatter plot for points
    sc = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=freq, cmap=cm, marker='o', norm=norm)
    
    # Determine the plot limits
    xmax, xmin = np.max(points[:, 0]), np.min(points[:, 0])
    ymax, ymin = np.max(points[:, 1]), np.min(points[:, 1])
    zmax, zmin = np.max(points[:, 2]), np.min(points[:, 2])
    
    if xmax - xmin <= l0:
        xmax, xmin = l0, -l0
    if ymax - ymin <= l0:
        ymax, ymin = l0, -l0
    if zmax - zmin <= l0:
        zmax, zmin = l0, -l0
        
    Lmax = np.max([xmax, ymax])
    Lmin = np.min([xmin, ymin])
    ax.set_xlim(Lmin, Lmax)
    ax.set_ylim(Lmin, Lmax)
    ax.set_zlim( z0*1.1, zmax)

    # Plot vectors as arrows
    for point, vector in zip(points, vectors):
        ax.quiver(point[0], point[1], point[2], vector[0], vector[1], vector[2], length=l0 / 2., normalize=True)
    
    if np.abs(z0) < 10*Lmax:
        # Add a flat black surface at z = z0
        x_surf = np.linspace(Lmin, Lmax, 100)
        y_surf = np.linspace(Lmin, Lmax, 100)
        x_surf, y_surf = np.meshgrid(x_surf, y_surf)
        z_surf = np.full_like(x_surf, z0)
        ax.plot_surface(x_surf, y_surf, z_surf, color='yellow', alpha=0.5)  
    else:
        Lmax = np.max([xmax, ymax, zmax])
        Lmin = np.min([xmin, ymin, zmin])
        ax.set_xlim(Lmin, Lmax)
        ax.set_ylim(Lmin, Lmax)
        ax.set_zlim(Lmin, Lmax)

    ax.set_xlabel('X - nm')
    ax.set_ylabel('Y - nm')
    ax.set_zlabel('Z - nm')
    
    # Add color bar to indicate the frequency values
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Frequency - GHz')

    plt.title(f'Positions of the {len(points)}-TLS with relative frequency and dipole direction')
    plt.show()
def plot_points_with_vectors_browser(points, vectors, l0):
    trace_points = go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(size=2, color='blue'),
        name='Points'
    )

    traces = [trace_points]

    for point, vector in zip(points, vectors):
        trace_vector = go.Cone(
            x=[point[0]],
            y=[point[1]],
            z=[point[2]],
            u=[l0*vector[0]],
            v=[l0*vector[1]],
            w=[l0*vector[2]],
            sizemode='scaled',
            sizeref=0.1,
            anchor='tail',
            colorscale='Blues',
            showscale=False
        )
        traces.append(trace_vector)

    layout = go.Layout(
        title='3D Scatter Plot of Points with Vectors',
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis'
        )
    )

    fig = go.Figure(data=traces, layout=layout)
    plot(fig)    

In [215]:
@njit
def solid_angle_averaging(I, Mprovv, theta, phi, limit_angle, dist):
    #avgI = I.copy()*0
    dphi = np.pi/phi.size
    dtheta = 2*np.pi/theta.size
    for n2 in range(phi.size):
        for n1 in range(theta.size):
            theta0 = theta[n1]
            phi0 = phi[n2]
            X0 = np.array( [np.cos(theta0)*np.sin(phi0), np.sin(theta0)*np.sin(phi0), np.cos(phi0) ] )
            count = 0
            total_weight = 0
            for nph in range(phi.size):
                for nth in range(theta.size):
                    theta_n = theta[nth]
                    phi_n = phi[nph]
                    Xtp = np.array( [np.cos(theta_n)*np.sin(phi_n), np.sin(theta_n)*np.sin(phi_n), np.cos(phi_n) ] )
                    projection = np.dot(X0, Xtp)
                    if projection >= np.cos(limit_angle):
                        weight = np.sin(phi_n)
                        Mprovv[n1,n2,:,:] += I[nth,nph,:,:]*weight #* dphi*dtheta
                        total_weight += weight
                        count += 1
            if total_weight > 0:
                Mprovv[n1,n2,:,:] = Mprovv[n1,n2,:,:]   /total_weight
            
    
    return Mprovv * dist**2. #* (np.sin(2*limit_angle))

In [216]:
def spin_operators(N, Nex):
    arr_local_dim = (2.*np.ones(N_TLS)).tolist()
    sigma_enr = enr_destroy( arr_local_dim, Nex )
    sigma_ = []
    s_ = []
    sx = []
    sy = []
    sz = []
    for nt in range(N_TLS):
        sigma_.append( Qobj(sigma_enr[nt].data) )
        s_.append( .5*sigma_[nt] )
        sx.append( .25*( sigma_[nt] + sigma_[nt].dag() ) )
        sy.append( -1j*.25*( sigma_[nt] - sigma_[nt].dag() ) )
        sz.append( sigma_[nt].dag() * sigma_[nt] - .5  )

    return sx, sy, sz, s_

In [ ]:
# ======== Define spin operators in the truncated Hilber space
N_TLS = 4
Nex = 4
sx, sy, sz, s_ = spin_operators(N_TLS, Nex)
id_hilbert = Qobj( np.eye(len(s_[0].full() ) ) )

lam0 = 785 # ==== laser wavelength in nm
k0 = 2*np.pi/lam0       # ==== for the dyadic one should in principle use k including the detuning. But 1Ghz of detuning is 10^-7 in k, completely irrelevant
# ==== total volume of the nanocrystal in nm
Lx = lam0 * 4
Ly = lam0 * 3
Lz = lam0 /7

# ============ distance between TLS
l0 = 400 # in nm
# ==== define the positions of all TLS
R = ordered_chain(l0, N_TLS)
R[1][0] = R[0][0] + 40
R[2][0] = R[3][0] - 40
#R, N_TLS = uniform_random_lattice([-Lx/2., Lx/2., -Ly/2., Ly/2., -Lz/2., Lz/2.], l0, N_TLS, max_attempts=10000)
# ==== select orientation dipoles
direction_dipole = 0    # x==0, y==1, z==2
# ==== define dipole moment axis of each TLS
D = partially_random_dipoles(N_TLS, direction_dipole, 0.01)
#D = np.array([[1,0,0],[np.cos(np.pi/6),np.sin(np.pi/6),0]])

# ==== reference electric field at lambda0 with d = 13 D
Edd_ref = 0.0248 # kV/cm
# ==== include a golden substrate
img_active = 1  # ==== choose to use free space '==0' or half space '==1' dyadic Green function
z0 = - Lz/2. - 20     # ==== position in nm of the conductinc plate (only if img_active == 1)

# ==== in GHz - detuning of the molecules with respect external pump, molecules losses, dipole-dipole reference scale
Udd = 1.    # ==== coefficiente dimensionale dyadic in GHz -> Udd = e^2/epsilon0 k0^3 xi_d^2 ( e xi_d = dipole transition \approx 13 D )
Gamma0 = Udd/(3.*np.pi)     # ==== Single emitter diagonal spontaneous decay rate of the free space dyadic. Necessary to ensure stability.
OmR = Gamma0 * 0
OmR_incoh = Gamma0 * .1

gamma = 1 * Gamma0  # ==== homogeneous broadening, or Fourier limit linewidth
gamma_deph = 4. * Gamma0 # ==== pure dephasing rate (to be done properly I should include the detuning dependence that takes account of the phonons spectral density)
gamma_deph_coll = 0.0 * Gamma0

sigma_freqs = 2*np.pi * 0.0
#arr_freq_dis = sigma_freqs*np.random.normal(0.0, 1., size=N_TLS) #array that stores fluctuations of TLS freqs., sigma is in GHz
#arr_freq_dis = sigma_freqs*np.random.uniform(-1./2., 1./2., size=N_TLS)
arr_freq_dis = sigma_freqs*np.linspace(-1., 1., N_TLS)

plot_points_with_vectors_substrate(R, D, 20., arr_freq_dis, z0)

# ==== detuning with respect external laser
det = -2*np.pi*0.        #in Ghz

Edrive = np.array([0, 0, 0])
Edrive[direction_dipole] = 1
projE = np.matmul( D, Edrive )
dir_k = np.array([0, 0, -1])
if direction_dipole == 2:
    dir_k = np.array([1, 0, 0])

K = np.zeros(shape=(N_TLS, N_TLS), dtype=np.complex_)
K = interaction_matrix(R, D, k0, N_TLS, z0, img_active)

Gamma_tens = 2.*Udd*K.imag + (gamma + Gamma0)*np.eye( N_TLS )
ikvals, ikvecs = np.linalg.eig(Gamma_tens)
idx = np.argsort(ikvals)
ikvals = ikvals[idx]
ikvecs = ikvecs[:,idx]

dressed_jump = []
for nik in range(ikvals.size):
    for ntls in range(N_TLS):
        Gk = ikvals[nik]
        if Gk < 0:
            Gk = 0.
        dressed_jump.append( np.sqrt( .5*( Gk + 1j*0.0 ) ) * ikvecs[ntls,nik] * 2*s_[ntls] )
    #print(ikvecs[:,nik])

Heff = 0*sz[0] #+ OmR * projE[0] * 2*(s_[0]*np.exp(-1j*k0*np.dot(dir_k, R[0]) ) + s_[0].dag()*np.exp(+1j*k0*np.dot(dir_k, R[0]) ) )
H0 = 0*sz[0]
jump_deph_coll = 0*sz[0]
arr_cjump = []
for n1 in range(N_TLS):
    H0 += ( arr_freq_dis[n1] ) * 4*s_[n1].dag()*s_[n1]
    Heff += ( det + arr_freq_dis[n1] ) * 4*s_[n1].dag()*s_[n1] + OmR * projE[n1] * 2*(s_[n1]*np.exp(-1j*k0*np.dot(dir_k, R[n1]) ) + s_[n1].dag()*np.exp(+1j*k0*np.dot(dir_k, R[n1]) ) )
    arr_cjump.append( np.sqrt(gamma/2.)*2*s_[n1]  )
    arr_cjump.append( np.sqrt(OmR_incoh/2.)*2*s_[n1].dag()  )
    arr_cjump.append( np.sqrt(gamma_deph/2.)*4*s_[n1].dag()*s_[n1] )
    jump_deph_coll += np.sqrt(gamma_deph_coll/2.)*4*s_[n1].dag()*s_[n1]
    for n2 in range(N_TLS):
        Heff += - 1.*Udd*K[n1][n2].real * 4*s_[n1].dag()*s_[n2]
        H0 += - 1.*Udd*K[n1][n2].real * 4*s_[n1].dag()*s_[n2]
arr_cjump.append( jump_deph_coll )
arr_jump_tot = arr_cjump + dressed_jump



Ltot = liouvillian(Heff, arr_jump_tot)
rho_ss = steadystate(Ltot)




In [ ]:
H1ex = - 1.*Udd*K.real + np.diag(arr_freq_dis)
ekvals, ekvecs = np.linalg.eig(  H1ex )
idx = np.argsort(ekvals)
ekvals = ekvals[idx]
ekvecs = ekvecs[:,idx]

print(ekvals)
print(ekvecs)

In [ ]:
#Calcolating the electric field dyadic at a given distance, on the whole solid angle
Omega_solid = np.pi/30
print(np.sin(2*Omega_solid))
# Set the fixed radius
dist = lam0*1000.
theta = np.linspace( 0., 2.*np.pi, 140 )
phi = np.linspace(0, np.pi/2., 100)
polar_Ex, polar_Ey, polar_Ez = polar_matrix_system_dyadic_field_amplitude( R, D, k0, N_TLS, theta, phi, dist, z0, img_active )
@njit
def polar_intensity_matrix(matE, Ni, Nj, Nn, M):
    for i in range(Ni):
        for j in range(Nj):
            for n in range(Nn):
                for m in range(Nn):
                    M[i][j][n][m] = np.conj(matE[i][j][n]) * matE[i][j][m]
    return M
# Get the shape of E
Ni, Nj, Nn = polar_Ex.shape
# Initialize the output tensor M_ijnm
Mxx = np.zeros((Ni, Nj, Nn, Nn), dtype=complex)
Myy = np.zeros((Ni, Nj, Nn, Nn), dtype=complex)
Mzz = np.zeros((Ni, Nj, Nn, Nn), dtype=complex)
Ixx = polar_intensity_matrix(polar_Ex, Ni, Nj, Nn, Mxx) # I[theta, phi, N_TLS, N_TLS] = Matrici N nella nuova notazione, theta = 0...2\pi polar angle 
Iyy = polar_intensity_matrix(polar_Ey, Ni, Nj, Nn, Myy)
Izz = polar_intensity_matrix(polar_Ez, Ni, Nj, Nn, Mzz)

I = Ixx + Iyy + Izz

I[1:theta.size,0,:,:] = I[1:theta.size,0,:,:].copy()*0 #remove the Ntheta-copies of the north-pole
Mprovv = np.zeros((Ni, Nj, Nn, Nn), dtype=complex)
I_avg = solid_angle_averaging(I, Mprovv, theta, phi, Omega_solid, dist )

In [91]:
# Get the shape of the tensor
n, m, i_dim, j_dim = I_avg.shape

# Ensure that i_dim == j_dim, as this operation only makes sense for square matrices
assert i_dim == j_dim, "The last two dimensions must be the same size (i.e., a square matrix for each n, m)."

# Create a mask where i == j (diagonal elements)
mask = np.eye(i_dim, dtype=bool)

# Set the off-diagonal elements to 0 for each n, m
I_avg[:, :, ~mask] = 0

In [276]:
taus = np.linspace(0., 20., 100)
Nw = 200
wlist = 2*np.pi*np.linspace(-1.5, 1.5, Nw)
taulist = np.linspace(0, 400., Nw)/np.pi



avg_s = np.zeros(shape=( N_TLS ), dtype=np.complex_)
avg_1corr = np.zeros(shape=( N_TLS,N_TLS ), dtype=np.complex_)
avg_2corr_time = np.zeros(shape=( len(taus) ,N_TLS,N_TLS,N_TLS,N_TLS ), dtype=np.complex_)
G1_ = np.zeros(shape=(len(taulist), N_TLS, N_TLS), dtype=np.complex_)
spec = np.zeros(shape=(len(wlist), N_TLS, N_TLS), dtype=np.complex_)

ds_ = []
for n in range(N_TLS):
    avg_s[n] = expect( 2*s_[n], rho_ss )
    ds_.append( s_[n] - 0*avg_s[n]/2.*id_hilbert )

for n1 in range(N_TLS):
    for n2 in range(N_TLS):
        avg_1corr[n1][n2] = expect( 4*ds_[n1].dag()*ds_[n2], rho_ss )
        #spec[:,n1,n2] = spectrum(Heff, wlist, arr_jump_tot, 2*s_[n1].dag(), 2*s_[n2])
        C1 = correlation_2op_1t(Heff, rho_ss, taulist, arr_jump_tot, 2*ds_[n1].dag(), 2*ds_[n2], solver='me')#, reverse=True)
        G1_[:, n1,n2] = C1
        C1avg = np.sum(C1)/(len(C1))
        wlist, S1 = spectrum_correlation_fft(taulist, C1-C1avg)
        spec[:,n1,n2] = S1
        
        if Nex > 0:
            for n3 in range(N_TLS):
                for n4 in range(N_TLS):
                    avg_2corr_time[:, n1,n2,n3,n4] = correlation_3op_1t( Heff, rho_ss, taus, arr_jump_tot, 2*ds_[n1].dag(), 4*ds_[n2].dag()*ds_[n3], 2*ds_[n4] )
                    

In [ ]:
print(avg_s)
print(np.abs(avg_1corr) )

In [ ]:
SpecN = np.sum( I_avg[:,:,np.newaxis,:,:]*spec[np.newaxis,np.newaxis,:,:,:], axis=(3,4) )
Spec_om_provv = np.abs(SpecN[0,0,:].real)
#Spec_om_provv = np.abs(spec[:,0,0].real)# + spec[:,0,1].real + spec[:,1,1].real +spec[:,1,0].real)

n00 = np.argmin(np.abs(wlist-0))
wlist_provv = wlist
wlist_plot = np.delete(wlist_provv, n00)
Spec_om = np.delete(Spec_om_provv, n00)

max_S = np.amax(Spec_om)
print(max_S)
nulist = wlist_plot#/(2*np.pi)
#plt.plot(nulist, np.log(Spec_om/max_S)/np.log(10.) )
plt.plot(nulist, Spec_om/max_S )
min_nu = np.amin(nulist)
max_nu = np.amax(nulist)

plt.ylim((0,1))
#plt.ylim((-1.5,0))
#plt.xlim((min_nu,max_nu))
plt.xlim((-5.,5.))


font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
plt.xlabel(r'$\omega/(2\pi)$ - GHz', fontdict=font)
plt.ylabel(r'$S(\omega)$', fontdict=font)

plt.grid()
# Adjust the layout to prevent overlap
plt.gcf().set_size_inches(120 / 25.4, 80 / 25.4)
plt.tight_layout()
# Show the plot
plt.show()




In [279]:
N1 = np.abs(np.sum( I_avg[:,:, :, :] * avg_1corr[np.newaxis, np.newaxis, :,:], axis=(2,3) ))

G2 = np.sum( I_avg[:,:, np.newaxis, :, np.newaxis, np.newaxis, :] * avg_2corr_time[np.newaxis, np.newaxis, :, :, :, :, :], axis=(3, 6))
G2 = np.sum( I_avg[:,:, np.newaxis, :, :] * G2, axis=(3,4))

In [ ]:
#Obserable_1d_0deph = np.zeros(len( G2[0,0,:]))
if gamma_deph < 0.0000001:
        Obserable_1d = G2[0,0,:]/(N1[0,0]**2) 
else:
        Obserable_1d_0deph = G2[0,0,:]/(N1[0,0]**2) 
#Obserable_1d_0deph = avg_2corr_time[:,0,0,0,0]/(avg_1corr[0][0])**2.         
plt.plot(taus, np.abs(Obserable_1d_0deph), color='red' , label=r'$\gamma_{\phi}=$%.2f GHz' % (gamma_deph))
#plt.plot(taus, np.abs(Obserable_1d), color='blue', label=r'$\gamma_{\phi}$=0', linestyle='--')

#plt.plot(taulist, np.abs(G1_[:,0,1]/avg_1corr[0][1]), color='green' )

plt.plot(taus, np.ones(len(taus)), color='black', linestyle='--')

plt.ylim((0.,2))
plt.xlim((0,20))


font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
plt.xlabel(r'$\tau$ - ns', fontdict=font)
plt.ylabel(r'$g^{(2)}(\tau)$', fontdict=font)

plt.gcf().set_size_inches(170 / 25.4, 130 / 25.4)
plt.tight_layout()

plt.legend(loc='best')


#plt.savefig('g2_example_cohPump_Ntls%d_deph%.2f_Omr=G0_z0%.2f_l0%.2f_NA%.2f.pdf' % (N_TLS, gamma_deph/Gamma0, z0, l0, np.sin(2*Omega_solid) ))

plt.grid()

In [ ]:
Phi, Theta = np.meshgrid(phi, theta)

Obserable = np.abs( N1 )

minO = np.amin(Obserable)
maxO = np.amax(Obserable)*(1.01) 
vmin = 0
vmax = 1.

# Create a polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
contour_min = vmin
contour_max = vmax
contour_interval = (contour_max-contour_min)/1000
levels = np.arange(contour_min, contour_max, contour_interval)
c = ax.contourf(  Theta, Phi, Obserable/maxO, levels=levels, cmap='viridis', vmin=vmin, vmax=vmax)
# Add a color bar
fig.colorbar(c)
# Set labels and title
#ax.set_xlabel(r'$\theta$')
#ax.set_ylabel(r'$\phi$')
ax.set_title(r'Photon density')

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

plt.gcf().set_size_inches(170 / 25.4, 100 / 25.4)
plt.tight_layout()

#plt.savefig('half_dome_photoemission%.2f.png' % (gamma_deph))
plt.show()  # Prevent static figure from displaying

In [ ]:
Phi, Theta = np.meshgrid(phi, theta)

Obserable = np.abs( np.divide(G2[:,:,:],np.square(N1[:,:,np.newaxis])) )
#Obserable = np.abs( G2 )

minO = np.amin(Obserable[:,:,0])
maxO = np.amax(Obserable[:,:,0])
vmin = minO
vmax = maxO

# Create a polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
contour_min = vmin
contour_max = vmax
contour_interval = (contour_max-contour_min)/1000
levels = np.arange(contour_min, contour_max, contour_interval)
c = ax.contourf(  Theta, Phi, Obserable[:,:,0], levels=levels, cmap='viridis', vmin=vmin, vmax=vmax)
# Add a color bar
fig.colorbar(c)
# Set labels and title
#ax.set_xlabel(r'$\theta$')
#ax.set_ylabel(r'$\phi$')
ax.set_title(r'$g^{(2)}(\tau=0)$ ')

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

plt.gcf().set_size_inches(170 / 25.4, 100 / 25.4)
plt.tight_layout()

#plt.savefig('half_dome_g2%.2f.png' % (gamma_deph))
plt.show()  # Prevent static figure from displaying

In [835]:
#Video of the electric field amplitude on the solid angle as a function of the control parameter
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
Phi, Theta = np.meshgrid(phi, theta)

Obserable = np.abs( np.divide(G2[:,:,:],np.square(N1[:,:,np.newaxis])) )
#Obserable = np.abs( G2 )

minO = np.amin(Obserable[:,:,:])
maxO = np.amax(Obserable[:,:,:])
vmin = minO
vmax = maxO

# Create a polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
contour_min = vmin
contour_max = vmax
contour_interval = (contour_max-contour_min)/100
levels = np.arange(contour_min, contour_max + contour_interval, contour_interval)
c = ax.contourf(  Theta, Phi, Obserable[:,:,0], levels=levels, cmap='viridis', vmin=vmin, vmax=vmax)
# Add a color bar
fig.colorbar(c)
# Set labels and title
ax.set_xlabel(r'$\theta$')
ax.set_ylabel(r'$\phi$')
ax.set_title(r'$g^{(2)}(\tau)$ ')
plt.close(fig)  # Prevent static figure from displaying
def update_Esolid(frame):    
    # Clear the previous contour plots
    ax.clear()
    
    # Re-plot the updated contour plots
    #c = ax.contourf(Theta, Phi, np.log(arr_Esolid[frame]/max_logEsolid + 10.**(contour_min+2*contour_interval)), levels=levels, cmap='viridis', vmin=vmin, vmax=vmax)
    c = ax.contourf( Theta, Phi, Obserable[:,:,frame], levels=levels, cmap='viridis', vmin=vmin, vmax=vmax)
    
    # Update title for current frame
    ax.set_title(r'$g^{(2)}(\tau)$ ')
    
    return c.collections

# Create animation
fps = 10
ani_Esolid = FuncAnimation(fig, update_Esolid, frames=len(taus), blit=False, interval=1000/fps)


In [ ]:
from PIL import Image
# Save the animation as a GIF
gif_filename = 'polar_plot_animation.gif'
ani_Esolid.save(gif_filename, writer='pillow', fps=fps, dpi=80)


In [ ]:
# Display animation
HTML(ani_Esolid.to_jshtml())

In [273]:
#Calcolating the electric field dyadic in (x,z)-space
a = 6
xvec = np.linspace(-lam0*a, lam0*a, 100)
yvec = np.array([ np.amax(np.abs(R[:,1])) + lam0*2. ])
zvec = np.linspace(z0, lam0*a, 100)
arr_Ex, arr_Ey, arr_Ez, arr_E_drive = system_dyadic_field_amplitude( R, D, k0, dir_k, N_TLS, xvec, yvec, zvec, z0, img_active )

In [274]:
Eamp_drive = np.square(np.abs(np.matmul(arr_Ex, avg_s ) + Edrive[0]*arr_E_drive)) + np.square(np.abs(np.matmul(arr_Ey, avg_s ) + Edrive[1]*arr_E_drive)) + np.square(np.abs(np.matmul(arr_Ez, avg_s ) + Edrive[2]*arr_E_drive)) 
Eamp_drive = Eamp_drive.reshape( (zvec.size, xvec.size) )

Eamp = np.square(np.abs(np.matmul(arr_Ex, avg_s ))) + np.square(np.abs(np.matmul(arr_Ey, avg_s ))) + np.square(np.abs(np.matmul(arr_Ez, avg_s ))) 
Eamp = Eamp.reshape( (zvec.size, xvec.size) )


In [ ]:
#nz0 = np.argmin( np.abs(zvec-z0) )
#Eamp_spec_int[:,0:nz0] = 0

maxEmap = np.amax(Eamp_drive)
print(maxEmap)
# Initialize figure for Efield function animation
fig_E, ax_E = plt.subplots(dpi=120, figsize=(4, 4))
Z, X = np.meshgrid(zvec, xvec)

vmin = 0.
vmax = 1.000
Emap_plot = ax_E.pcolormesh(X, Z, Eamp_drive/maxEmap , cmap='gist_heat')#, alpha=1, vmin=vmin, vmax=vmax)

"""vmin = -5
vmax = 0.
Emap_plot = ax_E.pcolormesh(Z, X, np.log(Eamp_spec_int/maxEmap + .00000001)/np.log(10.), cmap='viridis', alpha=1, vmin=vmin, vmax=vmax)
contour_min = -7
contour_max = 0.
contour_interval = .3
levels = np.arange(contour_min, contour_max + contour_interval, contour_interval)
contours = ax_E.contour(Z, X, np.log(Eamp_spec_int/maxEmap + .00000001)/np.log(10.), levels=levels, colors='black', alpha=.5)"""

#contour_labels = ax_E.clabel(contours, inline=True, fontsize=8, fmt='%.1f')

fig_E.colorbar(Emap_plot)

ax_E.set_xlabel(r'$x$ - nm')
ax_E.set_ylabel(r'$z$ - nm')
ax_E.set_aspect('equal')

In [ ]:

maxE = np.amax(Eamp)
fig_E, ax_E = plt.subplots(dpi=120, figsize=(4, 4))
Z, X = np.meshgrid(zvec, xvec)
vmin = -3
vmax = 0
#Emap_plot = ax_E.pcolormesh(X, Z, np.log(Eamp/maxE)/np.log(10.), cmap='viridis', alpha=1, vmin=vmin, vmax=vmax)

contour_min = .0
contour_max = .3
contour_interval = .02
levels = np.arange(contour_min, contour_max + contour_interval, contour_interval)

contours = ax_E.contour(X, Z, Eamp/maxE, levels=levels, colors='black', alpha=.5)
#contour_labels = ax_E.clabel(contours, inline=True, fontsize=8, fmt='%.1f')

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
ax_E.set_xlabel(r'$x$ - nm', fontdict=font)
ax_E.set_ylabel(r'$z$ - nm', fontdict=font)
ax_E.set_aspect('equal')

plt.gcf().set_size_inches(170 / 25.4, 100 / 25.4)
plt.tight_layout()

#plt.savefig('contour_example.pdf')

plt.show()